In [3]:
from datetime import datetime

In [6]:
def convert_date(date_str):
    if date_str:
        try:
            # First, check if the date is in the YYYYMMDD format (no separators)
            if len(date_str) == 8 and date_str.isdigit():
                # Convert YYYYMMDD to DD.MM.YYYY
                date_str = f"{date_str[6:8]}.{date_str[4:6]}.{date_str[:4]}"
            
            # Try parsing known formats
            for fmt in ['%d.%m.%Y', '%d.%m.%y', '%d/%m/%Y', '%d-%m-%Y', '%Y-%m-%d', '%Y.%m.%d']:
                try:
                    date_obj = datetime.strptime(date_str, fmt)
                    return date_obj.strftime('%Y-%m-%d')  # Return in YYYY-MM-DD format
                except ValueError:
                    continue  # Try the next format
            
            # If no format matched, raise an error
            raise ValueError(f"Unsupported date format: {date_str}")
        except Exception as e:
            print(f"Error converting date: {date_str} - {e}")
    return None  # Return None if the date is invalid or empty

In [7]:
test_dates = [
    "20231219",    # Compact: YYYYMMDD
    "19.12.23",    # Dots: DD.MM.YY
    "19/12/2023",  # Slashes: DD/MM/YYYY
    "2023-12-19",  # Hyphens: YYYY-MM-DD
    "19-12-2023",  # Hyphens: DD-MM-YYYY
    "invalid_date" # Invalid
]

for date in test_dates:
    print(f"Input: {date} -> Output: {convert_date(date)}")


Input: 20231219 -> Output: 2023-12-19
Input: 19.12.23 -> Output: 2023-12-19
Input: 19/12/2023 -> Output: 2023-12-19
Input: 2023-12-19 -> Output: 2023-12-19
Input: 19-12-2023 -> Output: 2023-12-19
Error converting date: invalid_date - Unsupported date format: invalid_date
Input: invalid_date -> Output: None


In [8]:
from connections import get_mongo_client

In [9]:
client = client = get_mongo_client()
db = client['task']
task_collection = db['task']
query = {
    'data.katm_333.return.data.general_cbr.loans.loan': {'$exists': True}
    , 'number': {'$eq': 7090120} 
}
projection = {
    'number': 1, 
    'request.clientId': 1,
    'data.katm_333.return.data.general_cbr.loans.loan': 1

}

In [13]:
import pandas as pd

In [12]:
docs = task_collection.find(query, projection)

In [14]:
for document in docs:
    rows = []

    number = document.get('number')
    client_id = document.get('request', {}).get('clientId', {})
    loans = document.get('data', {}).get('katm_333', {}).get('return', {}).get('data', {}).get('general_cbr', {}).get('loans', {}).get('loan', [])       

    if isinstance(loans, dict):
        loans = [loans]

    for loan in loans:
        row = {'client_id': client_id, 'number': number, **loan}
        rows.append(row)
        
    df = pd.DataFrame(rows)

In [15]:
df

,client_id,number,ttl_delq_90_119,ttl_delq_8_29,other_past_due,collaterals,ttl_delq_60_89,type,uuid,initial_pmt,...,principal_past_due,relationship,fact_close_date,is_own,outstanding,next_pmt_principal_date,coborrowers_count,principal_repaid_amount,guarantors,inf_confirm_date
0,6854981,7090120,,,0,,,32,033-00223-1753248,104300,...,0,1,20.10.2022,1,0,,0,2000000,,20.10.2022
1,6854981,7090120,,,0,,,32,033-00223-1815967,66921,...,0,1,27.07.2022,1,0,,0,1300000,,27.07.2022
2,6854981,7090120,,,0,,,32,033-00223-2066974,54839,...,0,1,13.04.2023,1,0,,0,1400000,,13.04.2023
3,6854981,7090120,,,,,,32,033-00223-2077710,46703,...,0,1,17.07.2023,1,0,,0,1200000,,17.07.2023
4,6854981,7090120,,,,,,32,033-00223-2258249,116003,...,0,1,,1,1882653.07,04.11.2024,0,1117346.93,,10.10.2024
5,6854981,7090120,,,,,,32,033-00223-1976292,52541,...,0,1,17.07.2023,1,0,,0,1000000,,17.07.2023
6,6854981,7090120,,,,,,32,033-00223-2534886,81793,...,0,1,,1,1581116.97,08.11.2024,0,418883.03,,10.10.2024
7,6854981,7090120,,,,,,32,033-00223-2628865,97831,...,0,1,,1,2001112.46,01.11.2024,0,298887.54,,10.10.2024
8,6854981,7090120,,,,,,32,033-00223-2754352,148298,...,,1,12.08.2024,1,0,,0,3400000,,12.08.2024
9,6854981,7090120,,,,,,32,033-00223-2815488,89549,...,,1,,1,2100000,04.11.2024,0,0,,10.10.2024


In [1]:
import pandas as pd
from connections import get_mongo_client, get_sql_server_connection
from utils import load_columns, get_date_range_by_offset, max_number_find

In [2]:
client = get_mongo_client()